In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import sklearn
import seaborn as sns
import warnings
from sklearn.metrics import mean_squared_error
from sklearn import metrics
warnings.filterwarnings('ignore')

In [20]:
math_train = pd.read_csv('train.csv')

In [21]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [22]:
def generate_labelencoder(atts):
    for attr in atts:
        math_train[attr] = le.fit_transform(math_train[attr])
    return math_train

In [23]:
math_train = generate_labelencoder(['TP_SEXO', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'])

In [24]:
math_train['TP_ENSINO'].fillna(int(math_train['TP_ENSINO'].mode()), inplace = True)
math_train['TP_DEPENDENCIA_ADM_ESC'].fillna(int(math_train['TP_DEPENDENCIA_ADM_ESC'].mode()),inplace = True)
math_train['TP_STATUS_REDACAO'].fillna(int((math_train['TP_STATUS_REDACAO']).mode()),inplace = True)

math_train['NU_NOTA_CN'].fillna(np.mean(math_train['NU_NOTA_CN']),inplace = True)
math_train['NU_NOTA_CH'].fillna(np.mean(math_train['NU_NOTA_CH']),inplace = True)
math_train['NU_NOTA_LC'].fillna(np.mean(math_train['NU_NOTA_LC']),inplace = True)
math_train['NU_NOTA_MT'].fillna(np.mean(math_train['NU_NOTA_MT']),inplace = True)
math_train['NU_NOTA_REDACAO'].fillna(np.mean(math_train['NU_NOTA_REDACAO']),inplace = True)

In [25]:
x = math_train[['NU_NOTA_CN','NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO','NU_IDADE','TP_ENSINO','TP_DEPENDENCIA_ADM_ESC','TP_STATUS_REDACAO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'IN_TREINEIRO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_SEXO', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']]
y = math_train['NU_NOTA_MT']

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

In [27]:
from sklearn.linear_model import LinearRegression
reglin = LinearRegression()
reglin.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [28]:
y_prev1 = reglin.predict(x_test)

In [29]:
#RMSE
print(np.sqrt(metrics.mean_squared_error(y_test, y_prev1)))

64.4637409397477


In [30]:
math_test = pd.read_csv('test.csv')

In [31]:
math_test['TP_ENSINO'].fillna(int(math_test['TP_ENSINO'].mode()), inplace = True)
math_test['TP_DEPENDENCIA_ADM_ESC'].fillna(int(math_test['TP_DEPENDENCIA_ADM_ESC'].mode()),inplace = True)

math_test.update(math_test['NU_NOTA_CN'].fillna(0))
math_test.update(math_test['NU_NOTA_CH'].fillna(0))
math_test.update(math_test['NU_NOTA_LC'].fillna(0))
math_test.update(math_test['NU_NOTA_REDACAO'].fillna(0))
math_test.update(math_test['TP_STATUS_REDACAO'].fillna(0))

In [32]:
def generate_labelencoder(atts):
    for attr in atts:
        math_test[attr] = le.fit_transform(math_test[attr])
    return math_test

In [33]:
math_test = generate_labelencoder(['TP_SEXO', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'])

In [34]:
features_train = ['NU_NOTA_CN','NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO','NU_IDADE','TP_ENSINO','TP_DEPENDENCIA_ADM_ESC','TP_STATUS_REDACAO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'IN_TREINEIRO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_SEXO', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']
math_test[features_train]
y_prev2 = reglin.predict(math_test[features_train])
math_test['NU_NOTA_MT'] = y_prev2

In [35]:
answer = pd.DataFrame()
answer ['NU_INSCRICAO'] = math_test.NU_INSCRICAO
answer ['NU_NOTA_MT'] = y_prev2

In [36]:
for nota in answer ['NU_NOTA_MT']:
    if nota < 0:
        answer['NU_NOTA_MT'] = answer['NU_NOTA_MT'].replace(nota, 0)

In [37]:
#answer.to_csv('answer.csv', index = False)

In [38]:
from sklearn.ensemble import RandomForestRegressor
tree_rf = RandomForestRegressor()

In [39]:
model_rf = tree_rf.fit(x_train, y_train)

In [40]:
y_prev_rf1 = model_rf.predict(x_test)

In [41]:
#MSE
mean_squared_error(y_test, y_prev_rf1)

3862.9724441832345

In [42]:
#Score
model_rf.score(x_test, y_test)

0.5027466173405635

In [43]:
y_prev_rf2 = model_rf.predict(math_test[features_train])
math_test['NU_NOTA_MT'] = y_prev_rf2

In [44]:
answer = pd.DataFrame()
answer ['NU_INSCRICAO'] = math_test.NU_INSCRICAO
answer ['NU_NOTA_MT'] = y_prev_rf2

In [45]:
for nota in answer ['NU_NOTA_MT']:
    if nota < 0:
        answer['NU_NOTA_MT'] = answer['NU_NOTA_MT'].replace(nota, 0)

In [46]:
answer.to_csv('answer.csv', index = False)